# prepare diversity constraint, QI etc

In [39]:
5636-939*3-419-366-123

1911

In [40]:
299287-12405

286882

In [56]:
l = [[] for _ in range(5)]
l[1].append(1)
l

[[], [1], [], [], []]

In [183]:
def parse_single_constraint(s):
    '''
    Args: String constraint
    format as: s = "gen(male) [100, 1000]"
    Return:
        dict, with attr, val, lower_bound, up_bound as key
    '''
    import re
    m = re.match(r'(.*)\((.*)\)\s* \[(\d+),\s* (\d+)\]', s)
    if m:
        d = dict()
        d['attr'], d['val'], d['lower_bound'], d['up_bound'] = m.group(1), m.group(2),int(m.group(3)),int(m.group(4))
        return d
    else:
        return None

In [1]:
import re

line = "Cats are smarter than dogs"

matchObj = re.match( r'(.*) are (.*?) .*', line, re.M|re.I)

if matchObj:
   print("matchObj.group() : ", matchObj.group())
   print ("matchObj.group(1) : ", matchObj.group(1))
   print ("matchObj.group(2) : ", matchObj.group(2))

matchObj.group() :  Cats are smarter than dogs
matchObj.group(1) :  Cats
matchObj.group(2) :  smarter


In [18]:
s = "gen(male) [100, 1000]"
d = parse_single_constraint(s)
d

{'attr': 'gen', 'lower_bound': '100', 'up_bound': '1000', 'val': 'male'}

In [22]:
def parse_constraint_file(path):
    constraints = []
    with open(path) as f:
        lines = f.readlines()
        for l in lines:
            constraints.append(parse_single_constraint(l))
    return constraints
path = 'C:/Users/datasci/YuHuang/YuMcMasterDropbox/Dropbox/Huang_PaperReview/Huang_3rd_research/Huang_Data_Fairness/Huang_Data_Fairness_Experiment/Huang_code/constraint.txt'
l = parse_constraint_file(path)
l

[{'attr': 'city', 'lower_bound': '100', 'up_bound': '150', 'val': 'Rome'},
 {'attr': 'city', 'lower_bound': '100', 'up_bound': '150', 'val': 'Moscow'},
 {'attr': 'domain ',
  'lower_bound': '1000',
  'up_bound': '2500',
  'val': 'Institutions'},
 {'attr': 'domain ', 'lower_bound': '600', 'up_bound': '1500', 'val': 'Arts'},
 {'attr': 'country',
  'lower_bound': '100',
  'up_bound': '150',
  'val': 'United States'},
 {'attr': 'country',
  'lower_bound': '100',
  'up_bound': '150',
  'val': 'United Kingdom'},
 {'attr': 'gender', 'lower_bound': '100', 'up_bound': '200', 'val': 'male'}]

buildGraph 的作用是 find min Cover to satisfy the diversity constraint
也是和我slide中的思想一样, 给定一个node 代表 constraint, 只会影响它的 neighbor
lower_bound >= k for k-anonymity. 
先生成所有的可能的 X-group, 然后从中找一个可能的X-group 来满足 diversity constraint.

满足constraint的条件下, 每个group中的records个数应该越大越好

1. 根据constraint, 生成所有可能满足要求的clustering
2. 关键在于第二步, 如何得到 optimal clustering
3. 把得到的clustering 和 optimal clustering merge, 之后 apply k-anonymity


visit each node to satisfy lower bound first.
如果两个node 相连, 说明肯定有overlap在cluster上.

In [149]:
def check_valid_choice(node, visited, selected_choices, n):
    if node in visited:
        return
    if len(visited) == n: # find a solution
        return selected_choices
    visited.add(node)
    for c in node.possible_clusters:
        if compatible(c, selected_choices):
            selected_choices.add(c)
            for neighbor in node.neighbors:
                if neighbor not in visited:
                    check_valid_choice(neighbor, visited, selected_choices, n)
                    selected_choices.remove(c)

Here is the basic workflow of Mondrian:

1. Partition the raw dataset into k-groups using kd-tree. k-groups means that each group contains at least k records.
2. Generalization each k-group (Fig. 1(b)), such that each group has the same QID*.

In [174]:
class Node:
    '''
    One Node reprensent a constraint and the tuples
    that satify this constraints
    self.records is a list of tuples satisfy this constraint
    '''
    def __init__(self, id, constraint):
        self.id = id
        self.constraint = constraint
        # records satisfy the constraint
        # store records ids
        self.records = []
        self.neighbors = []
        
    def __str__(self):
        return 'Node id:{} , constraint:{}'.format(self.id, str(self.constraint))
        
    def __repr__(self):
        return 'Node id:{} , constraint:{}'.format(self.id, str(self.constraint))
    
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        if isinstance(other, A):
            return (self.id == other.id)
        return NotImplemented

In [40]:
def build_constraint_node_list(path):
    node_li = []
    constraints_li = parse_constraint_file(path)
    for i in range(len(constraints_li)):
        node_li.append(Node(i,constraints_li[i]))

    return node_li
node_l = build_constraint_node_list(path)
node_l[1].constraint

{'attr': 'city', 'lower_bound': '100', 'up_bound': '150', 'val': 'Moscow'}

In [142]:
def build_graph(node_li):
    '''
    if two nodes have overlap on the satisfied records
    then add them to each other neighbor list
    '''
    # must clear before build, otherwise append duplicate when call build_graph
    # because neighobrs is a list, mutable object
    for node in node_li:
        node.neighbors.clear()
    for i in range(len(node_li)):
        for j in range(i+1, len(node_li)):
            node_i, node_j = node_li[i], node_li[j]
            overlap_records = set(node_i.records) & set(node_j.records)
            if overlap_records:
                node_i.neighbors.append(node_j)
                node_j.neighbors.append(node_i)

    return node_li

node_1 = Node(1,{})
node_1.records = [1,2,3]
node_2 = Node(2,{})
node_2.records = [2,4]
node_12 = [node_1, node_2]
node_new_12 = build_graph(node_12)
node_new_12[0].neighbors[0].id

2

In [125]:
selected_col_census = ['workclass', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
def get_single_col_statistic(df, selected_col):
    mapping = dict()
    for col in selected_col:
#         print(df[col].describe())
        mapping[col] = df[col].value_counts().to_dict()
#         print('='*40)
    return mapping

def get_two_attr_freq(df, two_cols):
    for col1, col2 in two_cols:
        print("="*40)
        print(df.groupby([col1, col2]).size().sort_values(ascending=False)[:20].sort_index())
        
def check_diversity_satisfication(df, constraint):
    '''
    Args: 
        df, relation R
        constraint, constraint in dict() format
        with key d['attr'], d['val'], d['lower_bound'], d['up_bound']
    Return:
        True/False
        the index of tuples satisfies given constraint
    '''
    d = constraint
    attr, val, low, up = d['attr'], d['val'], d['lower_bound'], d['up_bound']
    filtered_df = df[df[attr] == val]
    satisfied = False
    if int(low) <= len(filtered_df) <= int(up):
        satisfied = True
    row_index = filtered_df.index
    return satisfied, filtered_df

def check_diversity_satisfication_by_node(df, node):
    constraint = node.constraint
    satisfied, filtered_df = check_diversity_satisfication(df, constraint)
    node.records = filtered_df.index
    return satisfied, filtered_df, node

In [77]:
import pandas as pd
df = pd.read_csv("./Data/diversity_sample_data.csv", skipinitialspace=True)
df
# df.columns

,gender,city,country,occupation
0,male,Montreal,Canada,writer
1,male,Toronto,Canada,writer
2,female,Montreal,Canada,Musician
3,male,Edmonton,Canada,Architect
4,female,Ottawa,Canada,Actor
5,female,Kingston,Canada,Athlete
6,female,Calgary,Canada,Actor


In [76]:
df[(df['country'] == 'Canada') & (df['gender'] == 'female')]

,gender,city,country,occupation
2,female,Montreal,Canada,Musician
4,female,Ottawa,Canada,Actor
5,female,Kingston,Canada,Athlete
6,female,Calgary,Canada,Actor


In [120]:
path1= './Data/diversity_sample_constraint.txt'
sample_constraints_node_li = build_constraint_node_list(path1)
c = sample_constraints_node_li[0].constraint
print(c)
a, df_b = check_diversity_satisfication(df, c)
df_b

{'attr': 'gender', 'val': 'male', 'lower_bound': '1', 'up_bound': '3'}


,gender,city,country,occupation
0,male,Montreal,Canada,writer
1,male,Toronto,Canada,writer
3,male,Edmonton,Canada,Architect


In [134]:
check_diversity_satisfication_by_node(df, sample_constraints_node_li[0])[2].records

Int64Index([0, 1, 3], dtype='int64')

In [137]:
# 
for node in sample_constraints_node_li:
    check_diversity_satisfication_by_node(df, node)
    print(node, node.records)

Node id:0 , constraint:{'attr': 'gender', 'val': 'male', 'lower_bound': '1', 'up_bound': '3'} Int64Index([0, 1, 3], dtype='int64')
Node id:1 , constraint:{'attr': 'gender', 'val': 'female', 'lower_bound': '2', 'up_bound': '5'} Int64Index([2, 4, 5, 6], dtype='int64')
Node id:2 , constraint:{'attr': 'city', 'val': 'Toronto', 'lower_bound': '2', 'up_bound': '5'} Int64Index([1], dtype='int64')
Node id:3 , constraint:{'attr': 'occupation', 'val': 'writer', 'lower_bound': '3', 'up_bound': '5'} Int64Index([0, 1], dtype='int64')


In [163]:
sample_constraints_node_li = build_graph(sample_constraints_node_li)
sample_constraints_node_li[0].neighbors
index = sample_constraints_node_li[0].records
df.iloc[index] # tuples of clustering that satisfy constraint 0
# 可以用来得到node.record 中的index 转化为 实际records clustering

,gender,city,country,occupation
0,male,Montreal,Canada,writer
1,male,Toronto,Canada,writer
3,male,Edmonton,Canada,Architect


In [178]:
df.iloc[list(set(index))]

,gender,city,country,occupation
0,male,Montreal,Canada,writer
1,male,Toronto,Canada,writer
3,male,Edmonton,Canada,Architect


In [195]:
def bfs(node, visited, curr_clusters):
#     nonlocal curr_clusters
    q = [node]
    while q:
        node = q.pop(0)
        visited.add(node)
        # update current Node
        # 注意, 对 curr_records 修改, 并不会影响curr_clusters
        max_low, curr_records = curr_clusters
        if not curr_records:
            print(node.records)
            curr_records = list(node.records)[:]
            print(curr_records)
        inter = set(curr_records) & set(node.records)
        low_bound = int(node.constraint['lower_bound'])
        max_low = max(max_low, low_bound)
        if len(inter) >= max_low:
            curr_records = inter
        else:
            curr_records = list(set(curr_records) | set(node.records[:low_bound+1]))
        
        for neigh in node.neighbors:
            if neigh not in visited:
                q.append(neigh)
    return visited, curr_clusters

# curr_clusters[0] 是max low_bound, curr_clusters[1] 是 records index
curr_clusters = (0, []) 
visited = set()
for node in sample_constraints_node_li:
    if node not in visited:
        bfs(node, visited, curr_clusters)
        print(curr_clusters)

Int64Index([0, 1, 3], dtype='int64')
[0, 1, 3]
Int64Index([1], dtype='int64')
[1]
Int64Index([0, 1], dtype='int64')
[0, 1]
Int64Index([0, 1], dtype='int64')
[0, 1]
(0, [])
Int64Index([2, 4, 5, 6], dtype='int64')
[2, 4, 5, 6]
(0, [])


In [202]:
t = [0,[]]
a, b = t
b = t[1]
b = [1,2]
t

[0, []]

In [57]:
def compute_penalty(t, df, clusters):
    pen = 0
    for cluster in clusters:
        if t in cluster:
            if t.suppressed:
                t.penalty = len(df)
            else:
                t.penalty = len(cluster)
    return t